In [ ]:
%matplotlib inline

In [70]:
import matplotlib.pyplot as plt
import netCDF4
import numpy as np
import os
import geopandas as gpd
import pandas as pd
from garjmcmctdem_utils import aem_utils, spatial_functions
from garjmcmctdem_utils import plotting_functions as plots

In [74]:
# Import the points of interest

infile = "/media/nsymington/My Passport/Beetaloo/Beetaloo_extra_AEM_points.csv"
df = pd.read_csv(infile)

gdf = gpd.GeoDataFrame(
    df, geometry=gpd.points_from_xy(df.Long_GDA94,
                                    df.Lat_GDA94)).set_crs("EPSG:4283").to_crs("EPSG:28353")

gdf['easting'] = [gdf['geometry'].iloc[i].x for i in range(len(gdf))]
gdf['northing'] = [gdf['geometry'].iloc[i].y for i in range(len(gdf))]

In [76]:
aem_nc_path = "/home/nsymington/Documents/GA/AEM/AusAEM/galeisbs_vector_sum_point_located_NT/AusAEM_NT_galeisbstdem.nc"


aaem = aem_utils.AEM_inversion(name = 'AusAEM (galeisbstdem)',
                               inversion_type = 'deterministic',
                               netcdf_dataset = netCDF4.Dataset(aem_nc_path))

In [99]:
dist, ind = spatial_functions.nearest_neighbours(gdf[['easting', 'northing']].values,
                                                 np.column_stack((aaem.data['easting'][:],
                                                                  aaem.data['northing'][:])),
                                                 max_distance=10000)
line_inds = aaem.data['line_index'][ind].data

lines = aaem.data['line'][line_inds].data

gdf['ausaem_nn_ind'] = ind
gdf['ausaem_nn_dist'] = dist
gdf['line'] = lines

In [84]:
# Now we have the lines we can grid the lci conductivity data onto vertical grids (known as sections)
# this is the easiest way to visualise the AEM conuctivity in 2-dimensions

# Assign the lci variables to grid
grid_vars = ['conductivity', 'phid']

# Define the resolution of the sections
xres, yres = 80., 5.


# Define the output directory if saving the grids as hdf plots

grid_dir = "/media/nsymington/My Passport/Beetaloo/section_grids"

# if the directory doesn't exist, then create it
if not os.path.exists(grid_dir):
    os.mkdir(grid_dir)

aaem.grid_sections(variables = grid_vars, lines = np.unique(lines),
                      xres = xres, yres = yres,
                      return_interpolated = True, save_to_disk = True,
                      output_dir = grid_dir)



In [92]:
# panel 1 shows phid for a section.
def AusAEM_fid_plot(ax, cond, depth):
    cond_expanded = np.zeros(shape=2 * len(cond) + 1,
                                 dtype=np.float)

    cond_expanded[1:] = np.repeat(cond, 2)

    depth_expanded = (np.max(depth) + 10) * np.ones(shape=len(cond_expanded),
                                                            dtype=np.float)

    depth_expanded[:-1] = np.repeat(depth, 2)

    ax.plot(np.log10(cond_expanded), depth_expanded, c = 'k',
             linestyle = 'solid', label = 'AusAEM profile')
    ax.set_xticklabels([round(10 ** float(x), 4) for x in ax.get_xticks()])

    ax.set_ylabel('depth (mBGL)')
    ax.set_xlabel('Conductivity (S/m)')
    ax.grid(which = 'both')

    return ax

def dist_2_northticks(ax, section_data):
    north_ticks = ax.get_xticks().copy()

    for i, t in enumerate(north_ticks):
        mask = np.where(np.isclose(section_data['grid_distances'],t))[0]
        northing = section_data['northing'][mask]
        if len(northing) > 0:
            north_ticks[i]= northing[0]
        else:
            north_ticks[i]= np.nan

    ax.set_xticklabels(north_ticks)

def Aus_AEM_plot(point, bore = None, save_fig = True, fig_size = (10,6)):
    plt.close('all')

    fig = plt.figure(figsize = fig_size)

    ax1 = fig.add_axes([0.55, 0.62, 0.2, 0.3])
    ax2 = fig.add_axes([0.1, 0.4, 0.8, 0.1])
    ax3 = fig.add_axes([0.1, 0.1, 0.8, 0.3])
    ax4 = fig.add_axes([0.25, 0.62, 0.2, 0.3])

    cbar_ax = fig.add_axes([0.8, 0.6, 0.01, 0.3])

    cond = aaem.data['conductivity'][point]

    line_ind = aaem.data['line_index'][point]
    line = int(aaem.data['line'][line_ind].data)



    layer_top_depth = aaem.data['layer_top_depth'][point]

    fid = aaem.data['fiducial'][point]
    easting = aaem.data['easting'][point]
    northing = aaem.data['northing'][point]
    elevation = aaem.data['elevation'][point]

    ax1 = AusAEM_fid_plot(ax1, cond, layer_top_depth)

    ax1.set_ylim(plot_kwargs['panel_2']['max_depth'],0)


    #ax1.legend()
    res1 = plots.plot_single_line(ax2, aaem.section_data[line],
                                 'phid', plot_kwargs['panel_1'])

    ax2.set_title('AusAEM conductivity section - ' + str(line))

    dist = spatial_functions.xy_2_var(aaem.section_data[line],
                                      np.array([[easting, northing]]),
                                      'grid_distances')


    # Find distance along the lci section


    im2 = plots.plot_grid(ax3, aaem.section_data[line], 'conductivity',
                              panel_kwargs = plot_kwargs['panel_2'])

    ax3.plot([dist, dist], [-500, 500], 'pink')
    ax3.set_xlabel("Distance along line (m)")

    ax2.set_xlim(dist - plot_kwargs['panel_1']['buffer'],
                 dist + plot_kwargs['panel_1']['buffer'])
    ax3.set_xlim(dist - plot_kwargs['panel_2']['buffer'],
                 dist + plot_kwargs['panel_2']['buffer'])

    # For axis four we will create the AEM lines plotted

    for item in aaem.section_data.keys():
        if item == line:
            colour = 'blue'
        else:
            colour = 'k'
        ax4.plot(aaem.section_data[item]['easting'], aaem.section_data[item]['northing'],
                  c = colour)
    ax4.plot([easting], [northing], 'r+')

    cb = fig.colorbar(im2, cax=cbar_ax, orientation='vertical')

    cb.ax.set_yticklabels([round(10 ** x, 4) for x in cb.get_ticks()])
    cb.set_label('conductivity (S/m)', fontsize=10)

    ax_array = np.array([ax1,ax2,ax3,ax4])
    return fig, ax_array

In [93]:
# bring in the previously interpreted points

infile = "/media/nsymington/My Passport/Beetaloo/Beetaloo_base_cretaceous_interpreted.csv"

df_interp = pd.read_csv(infile)

df_interp

,fiducial,easting,northing,layer_elev,thickness,median_sig,max_sigma,min_sigma,line,distance_a,weighted_m,sigma_rati,geometry
0,6276.399902,181855.703125,8033348.5,162.330214,43.149782,0.068907,0.131747,0.026509,1010004,196920,0.079080,1.833497,POINT (181855.703125 8033348.5)
1,6296.399902,181834.593750,8034552.0,160.815876,44.324123,0.073612,0.178269,0.024908,1010004,195720,0.089051,3.093551,POINT (181834.59375 8034552)
2,6316.399902,181824.500000,8035741.5,160.815876,44.364117,0.089354,0.177754,0.033379,1010004,194520,0.103118,2.166140,POINT (181824.5 8035741.5)
3,6336.399902,181805.796875,8036925.0,159.901573,46.248420,0.066517,0.138140,0.013940,1010004,193340,0.077953,0.928527,POINT (181805.796875 8036925)
4,6356.399902,181785.093750,8038099.5,160.602150,45.987846,0.112596,0.186038,0.059540,1010004,192160,0.115670,2.701610,POINT (181785.09375 8038099.5)
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4311,5816.200195,489739.812500,8048227.0,198.053124,47.436881,0.016945,0.020779,0.014079,1160002,183880,0.017840,0.832168,POINT (489739.8125 8048227)
4312,5836.200195,489742.187500,8046874.0,196.241740,48.928258,0.055976,0.064104,0.031467,1160002,185220,0.048210,1.735347,POINT (489742.1875 8046874)
4313,5856.200195,489745.093750,8045527.0,193.389716,51.870278,0.092682,0.115726,0.056902,1160002,186580,0.082173,1.680669,POINT (489745.09375 8045527)
4314,5876.200195,489748.406250,8044236.0,193.389716,51.330285,0.066477,0.085524,0.047295,1160002,187860,0.061894,1.381410,POINT (489748.40625 8044236)


In [132]:
plot_kwargs = {'panel_1': {'color': 'black',
                      'ylabel': 'phid',
                      'legend': False,
                         'buffer': 10000.},
                  # panel 2 shows the ausaem conductivity section
                  'panel_2': {'title': 'LCI conductivity',
                      'max_depth': 200.,
                      'vmin': 0.001,
                      'vmax': 0.5,
                      'cmap': 'viridis',
                      'ylabel': 'elevation \n (mAHD)',
                       'buffer': 10000.,
                       'shade_doi': False}}

outdir = "/media/nsymington/My Passport/Beetaloo/plots"

for index, row in gdf.iterrows():
    point =row['ausaem_nn_ind']


    fig, ax_array = Aus_AEM_plot(point,
                                fig_size = (10,6))
    # scatter the interp point on ax1
    interp2section(ax_array[2], point, df_interp)

    ax_array[3].scatter([row['easting']],
                        [row['northing']],
                        c = 'pink', marker = 'o')

    plt.savefig(os.path.join(outdir, "{}_AusAEM_profile.png".format(row['Name'])),
                dpi = 200)
    plt.close()

# Plot our original points


/home/nsymington/anaconda3/envs/AEM_interp/lib/python3.7/site-packages/ipykernel_launcher.py:13: RuntimeWarning: divide by zero encountered in log10
  del sys.path[0]
/home/nsymington/anaconda3/envs/AEM_interp/lib/python3.7/site-packages/ipykernel_launcher.py:15: UserWarning: FixedFormatter should only be used together with FixedLocator
  from ipykernel import kernelapp as app
/home/nsymington/anaconda3/envs/AEM_interp/lib/python3.7/site-packages/ipykernel_launcher.py:105: UserWarning: FixedFormatter should only be used together with FixedLocator
/home/nsymington/anaconda3/envs/AEM_interp/lib/python3.7/site-packages/ipykernel_launcher.py:13: RuntimeWarning: divide by zero encountered in log10
  del sys.path[0]
/home/nsymington/anaconda3/envs/AEM_interp/lib/python3.7/site-packages/ipykernel_launcher.py:15: UserWarning: FixedFormatter should only be used together with FixedLocator
  from ipykernel import kernelapp as app
/home/nsymington/anaconda3/envs/AEM_interp/lib/python3.7/site-packa

In [123]:
def interp2section(ax, point, df):
    # mask interpreted points in dataframe
    line_ind = aaem.data['line_index'][point]
    line = int(aaem.data['line'][line_ind].data)

    df_line = df[df['line'] == line]
    dist_along_line = spatial_functions.xy_2_var(aaem.section_data[line],
                                                 df_line[['easting', 'northing']].values,
                                                 'grid_distances', max_distance = 10000.)

    # now scatter the interpreted points
    ax.scatter(dist_along_line,
              df_line['layer_elev'].values,
              c = 'k', marker = '+', s = 10)

In [117]:
df_interp.columns

Index(['fiducial', 'easting', 'northing', 'layer_elev', 'thickness',
       'median_sig', 'max_sigma', 'min_sigma', 'line', 'distance_a',
       'weighted_m', 'sigma_rati', 'geometry'],
      dtype='object')